# Analysis of raw non-zero suppressed data
### Non-zero suppressed runs: 8088, 8089

In [ ]:
from invisible_cities.cities import components as cp

import numpy as np
import matplotlib.pyplot as plt

### Variables to grab runs

In [ ]:
num_files = 3
run_number = 8088
num_plots = 3
data_dir = 'data/trigger1/'+str(run_number)+'/'
file_start = 'run_'+str(run_number)+'_000'
file_end = '_trigger1_waveforms.h5'
files = [data_dir+file_start+str(i+1)+file_end for i in range(0,num_files)]

### Print File Format

In [ ]:
i = 0
wfs = cp.wf_from_files(files, cp.WfType.rwf)
try:
    while wfs and i < 1:
        thisdata = next(wfs)
        keys = thisdata.keys()
        print('Keys:', keys)
        for key in keys:
            print(key, type(thisdata[key]))
            print('    ', np.shape(thisdata[key]))
            print('              ', thisdata[key])
        i += 1
except StopIteration:
    pass
finally:
    del wfs

### Collect data into numpy array

In [ ]:
i = 0
events = []
wfs = cp.wf_from_files(files, cp.WfType.rwf)
try:
    while wfs:
        thisdata = next(wfs)
        events.append(thisdata['sipm'])
        i += 1
except StopIteration:
    pass
finally:
    del wfs
events = np.array(events)
print('Number of Events: '+str(len(events)))

In [ ]:
# Plotting a few raw waveforms, without any corrections
for event in range(0,num_plots):
    summed_sipms = np.sum(events[event], axis=1)
    for sipm in events[event]:
        plt.plot(sipm)
    plt.xlabel(r'time bin [$\mu$s]')
    plt.ylabel('SiPM signal [ADC]')
    plt.title('Raw waveforms, uncorrected, Run '+str(run_number)+', Event '+str(event))
    #plt.xlim(640,660)
    #plt.ylim(60000,100000)
    plt.show()

### Finding the noisy SiPMs

In [ ]:
pedestals = np.mean(events, axis=2) # shape: (events, sipms)
event_std = np.std(events, axis=2) # shape: (events, sipms)

In [ ]:
plt.plot(np.mean(pedestals, axis=0))
plt.xlabel('SiPM')
plt.ylabel('Mean signal')
plt.show()

plt.plot(np.mean(event_std, axis=0))
plt.xlabel('SiPM')
plt.ylabel('mean std of events')
plt.show()

In [ ]:
bad_sipms = []
for event in range(0,len(events)):
    for sipm in range(len(events[event])):
        mean = np.mean(events[event][sipm])
        std = np.std(events[event][sipm])
        if mean > 70 or std > 70:
            bad_sipms.append(sipm)

worst_sipms = []
for sipm in np.unique(bad_sipms):
    count = np.count_nonzero(bad_sipms == sipm)
    print('SiPM '+str(sipm)+' suspicious in '+str(count)+' events')
    if count == len(events):
        worst_sipms.append(sipm)
print(worst_sipms)

In [ ]:
# Plotting a few raw waveforms, without any corrections
for event in range(0,num_plots):
    for sipm in range(len(events[event])):
        if sipm not in worst_sipms:
            plt.plot(events[event][sipm])
    plt.xlabel(r'time bin [$\mu$s]')
    plt.ylabel('SiPM signal [ADC]')
    plt.title('Raw waveforms, uncorrected, Run '+str(run_number)+', Event '+str(event))
    #plt.xlim(640,660)
    #plt.ylim(60000,100000)
    plt.show()

### Find and remove pedestal

In [ ]:
#pedestals = np.mean(events, axis=2) # shape: (events, sipms)
events = events - pedestals[:,:,np.newaxis]

In [ ]:
# Plotting a few raw waveforms, pedestal subtracted
for event in range(0,num_plots):
    for sipm in range(len(events[event])):
        if sipm not in worst_sipms:
            plt.plot(events[event][sipm])
    plt.xlabel(r'time bin [$\mu$s]')
    plt.ylabel('SiPM signal [ADC]')
    plt.title('Raw waveforms, pedestal subtracted, Run '+str(run_number)+', Event '+str(event))
    #plt.xlim(640,660)
    #plt.ylim(60000,100000)
    plt.show()

### Determine single pe calibration

In [ ]:
# Plotting a few raw waveforms, pedestal subtracted
for event in range(0,num_plots):
    for sipm in range(0,10):
        if sipm not in worst_sipms:
            plt.plot(events[event][sipm], 'o')
    plt.xlabel(r'time bin [$\mu$s]')
    plt.ylabel('SiPM signal [ADC]')
    plt.title('Raw waveforms, pedestal subtracted, Run '+str(run_number)+', Event '+str(event))
    plt.xlim(0,10)
    #plt.ylim(-10,10)
    plt.show()
    print(events[event][sipm])